In [94]:
# Импортирование основных библиотек

! pip install gensim
! pip install gensim --upgrade
import pandas as pd, os
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import word2vec,KeyedVectors
from gensim import downloader
import numpy as np
import datasets
import gensim
import nltk

In [95]:
# Импортирование датасета

dataset = pd.read_csv('/kaggle/input/product-item-data/sample-data.csv')
dataset

,id,description
0,1,Active classic boxers - There's a reason why o...
1,2,Active sport boxer briefs - Skinning up Glory ...
2,3,Active sport briefs - These superbreathable no...
3,4,"Alpine guide pants - Skin in, climb ice, switc..."
4,5,"Alpine wind jkt - On high ridges, steep ice an..."
...,...,...
495,496,Cap 2 bottoms - Cut loose from the maddening c...
496,497,Cap 2 crew - This crew takes the edge off fick...
497,498,All-time shell - No need to use that morning T...
498,499,All-wear cargo shorts - All-Wear Cargo Shorts ...


In [96]:
# Выделение из датасета колонки с текстами

train_text_df = pd.DataFrame({'text': dataset['description']})
train_text_df.head()

,text
0,Active classic boxers - There's a reason why o...
1,Active sport boxer briefs - Skinning up Glory ...
2,Active sport briefs - These superbreathable no...
3,"Alpine guide pants - Skin in, climb ice, switc..."
4,"Alpine wind jkt - On high ridges, steep ice an..."


In [97]:
# Приготовление эмбеддингов tf_idf

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english', binary=True, max_features=25_000)
text_embeddings_tf_idf = tfidf.fit_transform(train_text_df['text'] ).toarray()
text_embeddings_tf_idf

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [98]:
embeddings = gensim.downloader.load('word2vec-google-news-300')

In [99]:
text_embeddings_word2vec = []
stopwords = nltk.corpus.stopwords.words('english')
for doc in train_text_df['text'].str.lower().str.replace('[a-z]', ''):
    temp = []
    for word in doc.split(' '):
        if word not in stopwords:
            try:
                word_vec = embeddings[word]
                temp.append(word_vec)
            except:
                pass
    doc_vector = sum(temp)/len(temp)
    text_embeddings_word2vec.append(doc_vector)
len(text_embeddings_word2vec)

500

In [100]:
# Матрица косинусных расстояний для эмбеддингов tf_idf

array = sklearn.metrics.pairwise.cosine_similarity(text_embeddings_tf_idf, Y=None, dense_output=True)
array

array([[1.        , 0.18652085, 0.11989642, ..., 0.04386885, 0.09243105,
        0.07568186],
       [0.18652085, 1.        , 0.43452928, ..., 0.0407376 , 0.12165498,
        0.05534363],
       [0.11989642, 0.43452928, 1.        , ..., 0.02476351, 0.05161329,
        0.03228341],
       ...,
       [0.04386885, 0.0407376 , 0.02476351, ..., 1.        , 0.04610441,
        0.0454197 ],
       [0.09243105, 0.12165498, 0.05161329, ..., 0.04610441, 1.        ,
        0.31195031],
       [0.07568186, 0.05534363, 0.03228341, ..., 0.0454197 , 0.31195031,
        1.        ]])

In [101]:
# Матрица косинусных расстояний для эмбеддингов word2vec

array2 = sklearn.metrics.pairwise.cosine_similarity(text_embeddings_word2vec, Y=None, dense_output=True)
array2

array([[0.9999997 , 0.9032341 , 0.8529645 , ..., 0.8572532 , 0.82991874,
        0.85092926],
       [0.9032341 , 1.0000005 , 0.9238016 , ..., 0.86417127, 0.84152335,
        0.8347699 ],
       [0.8529645 , 0.9238016 , 1.0000001 , ..., 0.85212815, 0.8097455 ,
        0.8188008 ],
       ...,
       [0.8572532 , 0.86417127, 0.85212815, ..., 0.9999999 , 0.8340199 ,
        0.8108154 ],
       [0.82991874, 0.84152335, 0.8097455 , ..., 0.8340199 , 1.0000002 ,
        0.9040051 ],
       [0.85092926, 0.8347699 , 0.8188008 , ..., 0.8108154 , 0.9040051 ,
        0.99999976]], dtype=float32)

In [102]:
# Уровень значимости косинусного расстояния для эмбеддингов tf_idf: 0.4

level = 0.4

In [103]:
similar_text_list_tf_idf = []
for i in range(len(array)):
    lst_a = []
    for j in range(i,len(array)):
        x = array[i][j]
        if x > level:
            lst_a.append(j+1) # ID
    similar_text_list_tf_idf.append(lst_a)

similar_text_list_tf_idf

[[1],
 [2, 3],
 [3],
 [4, 159],
 [5, 308],
 [6],
 [7],
 [8, 219, 220, 254, 255, 261, 262, 291],
 [9],
 [10],
 [11],
 [12],
 [13],
 [14],
 [15, 16, 480, 484, 485],
 [16, 485],
 [17],
 [18, 21, 171, 172, 341, 487],
 [19, 494],
 [20, 21, 172, 340, 341, 487, 488],
 [21, 171, 172, 340, 341, 487, 488],
 [22, 23, 174, 175, 359, 360, 496, 497],
 [23, 24, 174, 175, 359, 360, 496, 497],
 [24, 175, 277, 440, 441, 442, 443],
 [25, 176, 277, 446],
 [26, 36, 68, 98, 491],
 [27, 28, 452, 453],
 [28, 452, 453],
 [29],
 [30],
 [31],
 [32, 462, 463],
 [33],
 [34],
 [35, 179, 282, 473],
 [36, 491],
 [37, 481, 482],
 [38],
 [39],
 [40],
 [41],
 [42, 421],
 [43, 427],
 [44, 427],
 [45],
 [46, 409],
 [47],
 [48, 69, 238, 319, 332, 371, 396, 490],
 [49, 134],
 [50, 439],
 [51],
 [52, 444],
 [53, 342],
 [54],
 [55],
 [56, 397],
 [57],
 [58, 63, 64, 65, 432, 433, 467, 470, 476],
 [59],
 [60],
 [61],
 [62],
 [63, 64, 65, 432, 433, 467, 470, 471, 476, 491],
 [64, 65, 432, 433, 467, 470, 476],
 [65, 73, 432, 433,

In [78]:
# Уровень значимости косинусного расстояния для эмбеддингов word2vec: 0.95

level2 = 0.95

In [104]:
# Cоздание списка похожих текстов для эмбеддингов word2vec. 
# В подсписках списка - тексты, похожие на соответствующий текст (идёт первым в подсписке, нумерация с 0).

similar_text_list_word2vec = []
for i in range(len(array2)):
    lst_a = []
    for j in range(i,len(array2)):
        x = array2[i][j]
        if x > level2:
            lst_a.append(j+1) # ID
    similar_text_list_word2vec.append(lst_a)

similar_text_list_word2vec

[[1],
 [2],
 [3],
 [4, 159],
 [5, 308],
 [6, 184],
 [7],
 [8, 220],
 [9],
 [10],
 [11],
 [12],
 [13],
 [14],
 [15, 16, 480, 484, 485],
 [16, 480, 484, 485],
 [17],
 [18, 171],
 [19, 494],
 [20, 172, 340, 487, 488],
 [21, 172, 341, 488],
 [22, 23, 174, 175, 359, 360, 496, 497],
 [23, 174, 175, 359, 360, 497],
 [24, 441, 443],
 [25, 176, 277, 446],
 [26],
 [27, 28, 452, 453],
 [28, 452, 453],
 [29, 454],
 [30],
 [31, 437],
 [32, 462, 463],
 [33, 51],
 [34],
 [35, 282],
 [36],
 [37, 481, 482],
 [38],
 [39],
 [40],
 [41],
 [42, 421],
 [43, 427],
 [44, 427],
 [45],
 [46, 409],
 [47],
 [48, 69, 238, 319, 396, 490],
 [49, 134],
 [50, 439],
 [51, 129],
 [52, 444],
 [53, 342],
 [54],
 [55],
 [56],
 [57],
 [58, 63, 64, 65, 432, 433],
 [59],
 [60],
 [61],
 [62],
 [63, 64, 65, 432, 433],
 [64, 65, 432, 433],
 [65, 432, 433],
 [66],
 [67],
 [68],
 [69, 238, 319, 332, 396, 490],
 [70],
 [71],
 [72, 334],
 [73, 239, 335],
 [74],
 [75, 77, 361],
 [76, 125],
 [77, 361],
 [78],
 [79, 80],
 [80],
 [81, 4

In [105]:
with open("Predictions.txt", "w") as file:
    file.write("TF_IDF PREDICTIONS:")
    file.write('\n')
    for i in similar_text_list_tf_idf:
        file.write(str(i))
        file.write('\n')
    file.write('\n')
    file.write("WORD2VEC PREDICTIONS:")
    file.write('\n')
    for i in similar_text_list_word2vec:
        file.write(str(i))
        file.write('\n')